<a href="https://colab.research.google.com/github/ishandahal/stats453-deep_learning_torch/blob/main/perceptron_adaline_linear_regression/Grad_intermediate_var.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Intermediate Gradients in PyTorch via autograd's ```grad```**

In [ ]:
import torch
from torch.autograd import grad
import torch.nn.functional as F

In [ ]:
x = torch.tensor([3.], requires_grad=True)
w = torch.tensor([2.], requires_grad=True)
b = torch.tensor([1.], requires_grad=True)

u = x * w
v = u + b
a = F.relu(v)

d_a_b = grad(a, b, retain_graph=True)
d_a_u = grad(a, u, retain_graph=True)
d_a_v = grad(a, v, retain_graph=True)
d_a_w = grad(a, w, retain_graph=True)
d_a_x = grad(a, x, retain_graph=False)

for name, grad in zip("xwbuv", (d_a_x, d_a_w, d_a_b, d_a_u, d_a_v)):
    print("d_a_" + name, grad)

d_a_x (tensor([2.]),)
d_a_w (tensor([3.]),)
d_a_b (tensor([1.]),)
d_a_u (tensor([1.]),)
d_a_v (tensor([1.]),)


More efficient implementation of above

In [ ]:
x_ = torch.tensor([3.], requires_grad=True)
w_ = torch.tensor([2.], requires_grad=True)
b_ = torch.tensor([1.], requires_grad=True)

u_ = x_ * w_
v_ = u_ + b_
a_ = F.relu(v_)

partial_derivatives = grad(a_, (x_, w_, b_, u_, v_))

for name, grad in zip("xwbuv", (partial_derivatives)):
    print("d_a_" + name, grad)

d_a_x tensor([2.])
d_a_w tensor([3.])
d_a_b tensor([1.])
d_a_u tensor([1.])
d_a_v tensor([1.])


####**Intermediate Gradients in PyTorch via ```retain_grad```**

In [ ]:
import torch
from torch.autograd import grad
import torch.nn.functional as F

x_ = torch.tensor([3.], requires_grad=True)
w_ = torch.tensor([2.], requires_grad=True)
b_ = torch.tensor([1.], requires_grad=True)

u_ = x_ * w_
v_ = u_ + b_
a_ = F.relu(v_)

u_.retain_grad()
v_.retain_grad()

a_.backward()

for name, var in zip("xwbuv", (x_, w_, b_, u_, v_)):
    print(f"d_a_{name}", var.grad)

d_a_x tensor([2.])
d_a_w tensor([3.])
d_a_b tensor([1.])
d_a_u tensor([1.])
d_a_v tensor([1.])


### **Intermediate Gradients in PyTorch Using Hooks**

In [ ]:
import torch 
import torch.nn.functional as F

grads = {}
def save_grad(name):
    def hook(grad):
        grads[name] = grad
    return hook

x = torch.tensor([3.], requires_grad=True)
w = torch.tensor([2.], requires_grad=True)
b = torch.tensor([1.], requires_grad=True)

u = x * w
v = u + b

x.register_hook(save_grad('d_a_x'))
w.register_hook(save_grad('d_a_w'))
b.register_hook(save_grad('d_a_b'))
u.register_hook(save_grad('d_a_u'))
v.register_hook(save_grad('d_a_v'))

a = F.relu(v)

a.backward()

grads

{'d_a_b': tensor([1.]),
 'd_a_u': tensor([1.]),
 'd_a_v': tensor([1.]),
 'd_a_w': tensor([3.]),
 'd_a_x': tensor([2.])}